In [1]:
import pandas as pd
from colorama import Fore, Style
from datetime import date
from tabulate import tabulate
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors

def read_grocery_data(filename):
    """
    Read grocery data from a CSV file.
    """
    grocery_data = pd.read_csv(filename)
    return grocery_data

def write_grocery_data(grocery_data, filename):
    """
    Write grocery data to a CSV file.
    """
    grocery_data.to_csv(filename, index=False)

def display_menu(grocery_data):
    """
    Display the grocery menu with colorful and formatted headers.
    """
    print(Fore.YELLOW + Style.BRIGHT)
    print("\nItem Number   Item Name    Quantity       Unit Price")
    print("--------------------------------------------------------------")
    print(Style.RESET_ALL)  # Reset colors and formatting

    for index, row in grocery_data.iterrows():
        print(f"{row['item_no']}{' '*(15-len(str(row['item_no'])))}"
              f"{row['item_name']}{' '*(15-len(str(row['item_name'])))}"
              f"{row['quantity']}{' '*(15-len(str(row['quantity'])))}"
              f"{row['unit_price']:.2f}")

def add_item(grocery_data):
    """
    Add an item to the grocery list.
    """
    item_no = input("Enter Item No: ")
    item_name = input("Enter Item Name: ")
    quantity = int(input("Enter Quantity: "))
    unit_price = float(input("Enter Unit Price: "))
    new_item = {'item_no': [item_no], 'item_name': [item_name], 'quantity': [quantity], 'unit_price': [unit_price]}
    new_item_df = pd.DataFrame(new_item)
    grocery_data = pd.concat([grocery_data, new_item_df], ignore_index=True)
    write_grocery_data(grocery_data, 'grocery.csv')
    print(Fore.GREEN + "Item added successfully." + Style.RESET_ALL)
    return grocery_data

def update_item(grocery_data):
    """
    Update an item in the grocery list.
    """
    item_no = input("Enter Item No to update: ")
    quantity = int(input("Enter new quantity: "))
    unit_price = float(input("Enter new unit price: "))
    grocery_data.loc[grocery_data['item_no'].astype(str).str.strip() == item_no, ['quantity', 'unit_price']] = quantity, unit_price
    write_grocery_data(grocery_data, 'grocery.csv')
    print(Fore.GREEN + "Item updated successfully." + Style.RESET_ALL)

def remove_item(grocery_data):
    """
    Remove an item from the grocery list.
    """
    item_no = input("Enter Item No to remove: ")
    grocery_data = grocery_data[grocery_data['item_no'].astype(str).str.strip() != item_no]
    write_grocery_data(grocery_data, 'grocery.csv')
    print(Fore.GREEN + f"Item {item_no} removed successfully." + Style.RESET_ALL)
    return grocery_data

def generate_receipt(cart, total_cost):
    """
    Generate a visually appealing receipt in a PDF file with a well-aligned table and colorful elements.
    """
    receipt_path = "receipt.pdf"
    c = canvas.Canvas(receipt_path, pagesize=letter)

    # Title
    c.setFillColorRGB(0.5, 0, 0.5)  # Purple color
    c.setFont("Helvetica-Bold", 16)
    c.drawString(100, 750, "*Billing*")

    # Shop name
    c.setFillColorRGB(0.5, 0, 0)  # Dark red color
    c.setFont("Helvetica-Bold", 16)
    c.drawString(100, 720, "------------WALMART------------")

    # Address, PinCode, and Phone Number
    c.setFillColorRGB(0, 0, 0)  # Black color
    c.setFont("Helvetica", 12)
    c.drawString(100, 700, "Address: Vidyapura, 5th Cross, Mysore")
    c.drawString(100, 680, "PinCode: 570001")
    c.drawString(100, 660, "Phone Number: 366936554135")

    # Date
    c.drawString(100, 640, f"Date: {date.today()}")

    # Receipt items
    headers = ["Item No", "Item Name", "Quantity", "Unit Price", "Total Price"]
    rows = [[item[0], item[1], item[2], item[3], item[2] * item[3]] for item in cart]
    table_data = [headers] + rows
    col_widths = [40, 120, 60, 80, 80]
    row_height = 20
    table = Table(table_data, colWidths=col_widths, rowHeights=row_height)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.gray),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('BOX', (0, 0), (-1, -1), 0.25, colors.black)
    ]))
    table.wrapOn(c, 800, 580)
    table.drawOn(c, 100, 580)

    # Total cost
    c.setFont("Helvetica-Bold", 14)
    c.drawString(100, 560, "Total Cost:")
    c.drawString(200, 560, str(total_cost))

    # Thank you message
    c.setFillColorRGB(1, 0, 1)  # Magenta color
    c.setFont("Helvetica", 12)
    c.drawString(100, 540, "Thank you for shopping with us! Visit again!")

    # Border
    c.setStrokeColorRGB(0, 0, 0)
    c.rect(90, 530, 500, 240, stroke=1, fill=0)

    c.save()

def buy_items(grocery_data):
    """
    Allow the customer to buy items from the grocery list.
    """
    cart = []
    total_cost = 0
    while True:
        display_menu(grocery_data)
        item_no = input("Enter the item number to add to cart (0 to stop): ").strip()
        if item_no == '0':
            break
        item = grocery_data[grocery_data['item_no'].astype(str).str.strip() == item_no]
        if item.shape[0] == 0:
            print("Invalid item number.")
            continue
        item = item.iloc[0]
        quantity = int(input(f"Enter the quantity of {item['item_name']}: "))
        if quantity > item['quantity']:
            print("Not enough stock available.")
            continue
        cart.append([item['item_no'], item['item_name'], quantity, item['unit_price']])
        total_cost += quantity * item['unit_price']
        grocery_data.loc[grocery_data['item_no'].astype(str).str.strip() == item_no, 'quantity'] -= quantity

    generate_receipt(cart, total_cost)
    grocery_data.to_csv('grocery.csv', index=False)
    print(Fore.BLUE +"Your receipt is saved in pdf, view receipt.pdf")
    print(Fore.GREEN + "Thank you for shopping with us! Visit again!")
    print(Style.RESET_ALL)  # Reset colors

def main():
    filename = 'grocery.csv'
    grocery_data = read_grocery_data(filename)

    while True:
        user_type = input("\nAre you an owner or a customer? (Enter '1' for owner / '2' for customer / '3' to exit): ")
        if user_type == '3':
            break

        if user_type == '1':
            print("\nWelcome, owner!")
            while True:
                print("\n1. Display Grocery Menu")
                print("2. Add Item")
                print("3. Update Item")
                print("4. Remove Item")
                print("5. Exit")
                choice = input("Enter your choice: ")
                if choice == '1':
                    display_menu(grocery_data)
                elif choice == '2':
                    grocery_data = add_item(grocery_data)
                elif choice == '3':
                    update_item(grocery_data)
                elif choice == '4':
                    grocery_data = remove_item(grocery_data)
                elif choice == '5':
                    break
                else:
                    print("Invalid choice!")
        elif user_type == '2':
            print("\nWelcome, customer!")
            buy_items(grocery_data)
        else:
            print("Invalid input! Please enter '1' for owner, '2' for customer, or '3' to exit.")

if __name__ == "__main__":
    main()



Are you an owner or a customer? (Enter '1' for owner / '2' for customer / '3' to exit): 1

Welcome, owner!

1. Display Grocery Menu
2. Add Item
3. Update Item
4. Remove Item
5. Exit
Enter your choice: 1


Item Number   Item Name    Quantity       Unit Price
--------------------------------------------------------------

1              chocolates     48             30.00
2              biscuits       30             10.00
3              lays           23             15.00
4              milk           42             27.00
5              bread          36             45.00
6              butter         64             45.00
7              cornflakes     47             122.00
8              ice cream      54             25.00
9              pen            49             26.00
10             cake           43             15.00
11             coke           45             20.00
12             Juice          33             25.00
13             mazaa          57             50.00
14           